## Install Libraries

In [0]:
%pip install -U -qqqq databricks-langchain langgraph==0.3.4 langchain mlflow 
%restart_python

## Initialize Global Settings

In [0]:
from databricks_langchain import ChatDatabricks
from databricks.sdk import WorkspaceClient
import os
from typing import Annotated, TypedDict, Literal
from langgraph.prebuilt import ToolNode
from langchain_core.tools import tool, StructuredTool
from langgraph.graph import START, StateGraph
from langgraph.graph.message import AnyMessage, add_messages

w = WorkspaceClient()

os.environ["DATABRICKS_HOST"] = w.config.host
os.environ["DATABRICKS_TOKEN"] = w.tokens.create(comment="for model serving", lifetime_seconds=1200).token_value

llm = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct")

## Test Prompt

In [0]:
llm.invoke("hello, world")

## Tools

In [0]:
@tool
def get_wheel_chair_accessibility(city: str, accessibility: str, business_category: str) -> str:
    """
    Searches the Nimble Google Maps dataset for places in the specified city that match the given accessibility feature and business category.
    The agent should provide the city name, specify 'wheelchair' or related accessibility terms for the accessibility parameter,
    and specify the type of business (e.g., restaurant, hotel) for the business_category parameter.
    Returns: up to 5 matching places as JSON.
    """
    query = f"""
    SELECT *
    FROM `dais-hackathon-2025`.nimble.dbx_google_maps_search_daily
    WHERE city = '{city}'
      AND exists(
        accessibility,
        a -> a.display_name ILIKE '%{accessibility}%'
      )
      AND exists(
        business_category_ids,
        category -> category ILIKE '%{business_category}%'
      )
    LIMIT 5;
    """
    print(f"Executing query: {query}")  # debug
    df = spark.sql(query).toPandas()
    return df.to_json(orient='records', indent=2)

In [0]:
# List of tools that will be accessible to the graph via the ToolNode
tools = [get_wheel_chair_accessibility]
tool_node = ToolNode(tools)

# This is the default state same as "MessageState" TypedDict but allows us accessibility to custom keys
class GraphsState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    # Custom keys for additional data can be added here such as - conversation_id: str

graph = StateGraph(GraphsState)

# Function to decide whether to continue tool usage or end the process
def should_continue(state: GraphsState) -> Literal["tools", "__end__"]:
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:  # Check if the last message has any tool calls
        return "tools"  # Continue to tool execution
    return "__end__"  # End the conversation if no tool is needed
# Core invocation of the model
def _call_model(state: GraphsState):
    messages = state["messages"]
    llm = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct").bind_tools(tools)
    response = llm.invoke(messages)
    return {"messages": [response]}  # add the response to the messages using LangGraph reducer paradigm

# Define the structure (nodes and directional edges between nodes) of the graph
graph.add_edge(START, "modelNode")
graph.add_node("tools", tool_node)
graph.add_node("modelNode", _call_model)

# Add conditional logic to determine the next step based on the state (to continue or to end)
graph.add_conditional_edges(
    "modelNode",
    should_continue,  # This function will decide the flow of execution
)
graph.add_edge("tools", "modelNode")

# Compile the state graph into a runnable object
graph_runnable = graph.compile()
def invoke_our_graph(st_messages):
    return graph_runnable.invoke({"messages": st_messages})

In [0]:
invoke_our_graph("get me wheelchair accessible resturants in San Francisco")